In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras import layers, Sequential, optimizers, losses

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train ,x_test = x_train.astype(np.float32) / 255., x_test.astype(np.float32) / 255.
train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(batchsz * 5).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batchsz)

In [2]:
self.encoder = Sequential([
    layers.Dense(256, activation = tf.nn.relu),
    layers.Dense(128, activation = tf.nn.relu),
    layers.Dense(h_dim)
])
self.decoder = Sequential([
    layers.Dense(128, activation = tf.nn.relu),
    layers.Dense(256, activation = tf.nn.relu),
    layers.Dense(784)
])

<module 'tensorflow' from '/Users/zhangyunfei_06/opt/anaconda3/lib/python3.8/site-packages/tensorflow/__init__.py'>

In [ ]:
clsss AE(keras.Model):
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = Sequential([
            layers.Dense(256, activation = tf.nn.relu),
            layers.Dense(128, activation = tf.nn.relu),
            layers.Dense(h_dim)
        ])
        self.decoder = Sequential([
            layers.Dense(128, activation = tf.nn.relu),
            layers.Dense(256, activation = tf.nn.relu),
            layers.Dense(784)
        ])
        
    def call(self, inputs, training = None):
        h = self.encoder(inputs)
        x_hat = self.decoder(h)
        return x_hat

In [ ]:
model = AE()
model.build(input_shape=(4, 784))
model.summary()
optimizer = optimizers.Adam(lr = lr)

for epoch in range(100):
    for step, x in enumerate(train_db):
        x = tf.reshape(x, [-1, 784])
        with tf.GradientTape() as tape:
            x_rec_logits = model(x)
            rec_loss = tf.nn.sigmoid_cross_entropy_with_logits(label=x, logits=x_rec_logits)
            rec_loss = tf.reduce_mean(rec_loss)
        grads = tape.gradient(rec_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 100 == 0:
            print(epoch, step, float(rec_loss))

In [ ]:
x = next(iter(test_db))
logits = model(tf.reshape(x, [-1, 784]))
x_hat = tf.sigmoid(logits)
x_hat = tf.reshape(x_hat, [-1, 28, 28])
x_concat = tf.concat([x[:50],x_hat[:50]], axis=0)
x_concat = x_concat.numpy() * 255.
x_concat = x_concat.astype(np.uint8)

def save_images(imgs, name):
    new_im = Image.new('L', (280, 280))
    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode = 'L')
            new_im.paste(im, (i, j))
            idnex += 1
    new_im.save(name)

save_images(x_concat, 'ae_images/rec_epoch_%d.png' % epoch)